In [10]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import joblib

# --- 1. VERİ YÜKLEME VE BİRLEŞTİRME ---
try:
    # Dosya 1
    df1 = pd.read_csv('Türkçe Tweetlerde Analiz(Etiketli).csv', encoding='utf-8')
    df1.dropna(subset=['Tweet'], inplace=True)
    map1 = {'Negatif': 0, 'Nötr': 1, 'Pozitif': 2}
    df1['Etiket_Sayisal'] = df1['Etiket'].map(map1)
    df1 = df1[['Tweet', 'Etiket_Sayisal']].rename(columns={'Tweet': 'Metin'})

    # Dosya 2
    df2 = pd.read_csv('sentimentSet.csv', encoding='utf-8')
    df2.dropna(subset=['tweets'], inplace=True)
    map2 = {-1: 0, 0: 1, 1: 2}
    df2['Etiket_Sayisal'] = df2['sentiment'].map(map2)
    df2 = df2[['tweets', 'Etiket_Sayisal']].rename(columns={'tweets': 'Metin'})

    # Birleştirme
    df_final = pd.concat([df1, df2], ignore_index=True)
    df_final = df_final.dropna(subset=['Etiket_Sayisal'])
    df_final['Etiket_Sayisal'] = df_final['Etiket_Sayisal'].astype(int)
    
    print(f"Toplam Veri Sayısı: {len(df_final)}")

except Exception as e:
    print(f"Hata oluştu: {e}")

# --- 2. TEMİZLİK VE DENGELEME ---
def temizle(text):
    text = str(text).lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    return text

df_final['Temiz_Metin'] = df_final['Metin'].apply(temizle)

# Dengeleme (Her sınıftan eşit al)
min_sayi = df_final['Etiket_Sayisal'].value_counts().min()
df_0 = df_final[df_final['Etiket_Sayisal'] == 0].sample(n=min_sayi, random_state=42)
df_1 = df_final[df_final['Etiket_Sayisal'] == 1].sample(n=min_sayi, random_state=42)
df_2 = df_final[df_final['Etiket_Sayisal'] == 2].sample(n=min_sayi, random_state=42)
df_dengeli = pd.concat([df_0, df_1, df_2])

print(f"Dengelendikten sonra her sınıf için sayı: {min_sayi}")

# --- 3. EĞİTİM ---
print("Model eğitiliyor...")
vectorizer = TfidfVectorizer(max_features=5000)
X_vec = vectorizer.fit_transform(df_dengeli['Temiz_Metin'])
y = df_dengeli['Etiket_Sayisal']

model = MultinomialNB()
model.fit(X_vec, y)

# --- 4. KAYDETME (CRITICAL STEP) ---
joblib.dump(model, 'final_model.pkl')
joblib.dump(vectorizer, 'final_vectorizer.pkl')

print("✅ İŞLEM TAMAM! 'final_model.pkl' ve 'final_vectorizer.pkl' dosyaları oluşturuldu.")

Toplam Veri Sayısı: 6766
Dengelendikten sonra her sınıf için sayı: 1015
Model eğitiliyor...
✅ İŞLEM TAMAM! 'final_model.pkl' ve 'final_vectorizer.pkl' dosyaları oluşturuldu.
